In [1]:
import matplotlib.pyplot as plt
import os
import random
import cv2
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import svm, metrics
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
winSize = (64, 128)
blockSize = (16, 16)
blockStride = (8, 8)
cellSize = (8, 8)
nbins = 9
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

In [3]:
pos_path = []
neg_path = []
for i in os.listdir('pedestrians128x64'):
    if i[0] != '.':
        pos_path.append('pedestrians128x64/' + i)
for i in os.listdir('pedestrians_neg'):
    if i[0] != '.':
        neg_path.append('pedestrians_neg/' + i)
pos_images = []
for pos_image_path in pos_path:
    pos_image = cv2.imread(pos_image_path)
    hog_array = hog.compute(pos_image)
    pos_images.append(hog_array)
pos_images = np.resize(pos_images, (len(pos_images), 3780))
pos_labels = np.ones(shape=pos_images.shape[0], dtype=np.int32)

neg_images = []
for neg_image_path in neg_path:
    neg_image = cv2.imread(neg_image_path)
    for i in range(2):
        for j in range(4):
            sub_image = neg_image[i * 128:(i + 1) * 128, j * 64:(j + 1) * 64]
            hog_array = hog.compute(sub_image)
            neg_images.append(hog_array)
neg_images = np.resize(neg_images, (len(neg_images), 3780))
neg_labels = -np.ones(shape=neg_images.shape[0], dtype=np.int32)

X = np.concatenate((pos_images, neg_images))
y = np.concatenate((pos_labels, neg_labels))
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

In [9]:
svc = svm.SVC(kernel='rbf', degree=2, C=1, max_iter=-1)
svc.fit(X_train, y_train)

SVC(C=1, degree=2)

In [10]:
def predict(img, op):
    print(img)

    test_image = cv2.imread(img)
    test_image = cv2.resize(test_image, (1024, 768))

    h, w, _ = test_image.shape
    stride = 16
    find_window = []
    find_coordinate = []
    for windowsize in range(1, 5):
        for ystart in range(0, h, stride*windowsize):
            for xstart in range(0, w, stride*windowsize):
                if ystart+windowsize*128>h or xstart+windowsize*64>w:
                    continue
                window = test_image[ystart:ystart+windowsize*128, xstart:xstart+windowsize*64]
                s_window = cv2.resize(window, (64, 128))
                hog_array = hog.compute(s_window).reshape(1, -1)
                pred = svc.predict(hog_array)
                if pred[0] == 1:
                    find_window.append(window)
                    find_coordinate.append([(xstart, ystart), (xstart+windowsize*64, ystart+windowsize*128)])
    print("Done")

    for c in find_coordinate:
        cv2.rectangle(test_image, c[0], c[1], (0,255,0))
    cv2.imwrite(op, test_image)

predict_path = []
output_path = []
for i in os.listdir('test'):
    if i[0] != '.':
        predict_path.append('test/' + i)
        output_path.append('output/' + i)

for i in range(len(predict_path)):
    predict(predict_path[i], output_path[i])


test/01.jpg
Done
test/02.jpg
Done
test/03.jpg
Done
test/04.jpg
Done
test/05.jpg
Done
test/06.jpg
Done
test/07.jpg
Done
test/08.jpg
Done
test/09.jpg
Done
test/10.jpg
Done
test/11.jpg
Done
test/12.jpg
Done
test/13.jpg
Done
test/14.jpg
Done
test/15.jpg
Done
test/16.jpg
Done
